# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [60]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [61]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [62]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [63]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [64]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [65]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [66]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [67]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [68]:
X.shape                        # 10 features

(532, 10)

In [69]:
y.shape                        # 1 target vector

(532, 1)

In [70]:
# Use 70% of the data for training and the remainder for testing
# YOUR CODE HERE!
split = int(0.7 * len(X))    # this defines the split of 70% of the entire length of the X data. X has 532 datapoints, so the first 372 datapoints will be assigned to training.
X_train = X[: split]         # this takes the whole X dataset (532 datapoints) from index numbers 0-371 and assigns to the training. It applies the split at index number 372, so anything from that index is not included in the training. 
X_test = X[split: ]          # Any X data from index number 372 and onward is assigned to test.                 
y_train = y[: split]
y_test = y[split : ]

In [71]:
from sklearn.preprocessing import MinMaxScaler

# CODER'S Note: Using SOlUTIONS cell for this segment, my original cell was not scaling the data between 0-1, and giving me vertical vectors of 3600, 3700, etc. These high values cannot be used for the model.

# Creating 4 MinMaxScaler() objects. 
X_train_scaler = MinMaxScaler()                         # create a scaler object
X_test_scaler = MinMaxScaler()
y_train_scaler = MinMaxScaler()
y_test_scaler = MinMaxScaler()

# fitting the train data
X_train_scaler.fit(X_train)                               # fit each scaler objects with the appropriate X,y train/test datasets
y_train_scaler.fit(y_train)
X_test_scaler.fit(X_test)
y_test_scaler.fit(y_test)

# transform the data from raw form to scaled form. 
X_train = X_train_scaler.transform(X_train)               # transform each X,y dataset into normalized values between 0-1. assign our scaler objects that are fitted with the raw dataset, and transform is run on those values. 
y_train = y_train_scaler.transform(y_train)
X_test = X_test_scaler.transform(X_test)
y_test = y_test_scaler.transform(y_test)

In [72]:
# Reshape the features for the model
# YOUR CODE HERE!
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))           # we do this because LSTM needs the features to be in a vertical vector. 
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


print(X_test[:5])       # Features values are normalized/scaled between 0-1, presented in a vertical vector.    

[[[0.01034043]
  [0.00242586]
  [0.00307681]
  [0.00183924]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.01974407]]

 [[0.00242586]
  [0.00307681]
  [0.00183924]
  [0.        ]
  [0.00051155]
  [0.00254834]
  [0.00577449]
  [0.        ]
  [0.02614593]
  [0.02101502]]

 [[0.00307681]
  [0.00183924]
  [0.        ]
  [0.00051155]
  [0.00305859]
  [0.00830812]
  [0.00577449]
  [0.02614593]
  [0.02740859]
  [0.02643977]]

 [[0.00183924]
  [0.        ]
  [0.00051155]
  [0.00305859]
  [0.00881542]
  [0.00830812]
  [0.03176945]
  [0.02740859]
  [0.03279791]
  [0.02240906]]

 [[0.        ]
  [0.00051155]
  [0.00305859]
  [0.00881542]
  [0.00881542]
  [0.03423683]
  [0.03302481]
  [0.03279791]
  [0.02879352]
  [0.02747223]]]


---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [74]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

# YOUR CODE HERE!
model = Sequential ()                            # build the RNN model

number_units = 30                                # 
dropout_fraction = 0.2                           # set the dropout rate to 20%, so after running through each layer, it keeps 80% of the data. 

# layer 1

model.add(LSTM(                                  # Define an LSTM layer. 
    units = number_units,                        # passing the units
    return_sequences =  True,                    # return sequences = True to connect to the next LSTM layer.
    input_shape = (X_train.shape[1], 1)          # X_train.shape[1] is number of time steps, 10 days worth of prices. The second 1 is number of input features it will use to predict 11th day price. 
))

model.add(Dropout(dropout_fraction))              # add first dropout layer, drops 20% 

# layer 2

model.add(LSTM(                                  # Make another LSTM layer.
    units = number_units,                          
    return_sequences = True                      # return_sequences =True to connect to the next LSTM layer. 
))

model.add(Dropout(dropout_fraction))             # 2nd dropout layer, drops another 20% 

# layer 3

model.add(LSTM(units=number_units))               # Define final LSTM layer. Only need to pass units param in 3rd LSTM layer
                                                  # No need to add a return_sequences param, no other LSTM will be added.

model.add(Dropout(dropout_fraction))              # dropout layer, drops 20%

# output layer 
model.add(Dense(1))                               # Dense output layer that should return only 1 final value. 

In [75]:
# Compile the model
# YOUR CODE HERE!
model.compile(optimizer='adam', loss='mean_squared_error')         

In [76]:
# Summarize the model
# YOUR CODE HERE!
model.summary()  

# 18,511 parameters. 
# All defined 3 LSTM layers, and 3 Droput layers are present. 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 30)            3840      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 30)            7320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_5 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [77]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
# YOUR CODE HERE!

model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)   # when working with Time Series data in NN's you have to set shuffle = False, otherwisw it will not be looking at the sequential data.

# CODER'S NOTE: Loss function begins at .0276 but adter 10 epoch is lowered to .0191, not that great of a decline, woud optimally like to see a loss close to .00191

Epoch 1/10
372/372 [==============================] - 12s 4ms/step - loss: 0.0276
Epoch 2/10
372/372 [==============================] - 2s 5ms/step - loss: 0.0238
Epoch 3/10
372/372 [==============================] - 2s 5ms/step - loss: 0.0287
Epoch 4/10
372/372 [==============================] - 2s 5ms/step - loss: 0.0286
Epoch 5/10
372/372 [==============================] - 2s 4ms/step - loss: 0.0241
Epoch 6/10
372/372 [==============================] - 2s 4ms/step - loss: 0.0221A: 0s - 
Epoch 7/10
372/372 [==============================] - 2s 4ms/step - loss: 0.0223
Epoch 8/10
372/372 [==============================] - 2s 4ms/step - loss: 0.0188
Epoch 9/10
372/372 [==============================] - 2s 4ms/step - loss: 0.0200
Epoch 10/10
372/372 [==============================] - 2s 5ms/step - loss: 0.0191


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [38]:
# Evaluate the model
# YOUR CODE HERE!
model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 2ms/step - loss: 0.0534


0.05337952449917793

In [78]:
# Make some predictions
# YOUR CODE HERE!
predicted = model.predict(X_test)

In [79]:
# Recover the original prices instead of the scaled version
predicted_prices = y_test_scaler.inverse_transform(predicted)
real_prices = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))

In [80]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.tail()

,Real,Predicted
2019-07-25,9772.139648,7155.851074
2019-07-26,9882.429688,7135.718262
2019-07-27,9847.450195,7106.458496
2019-07-28,9478.320313,7080.506836
2019-07-29,9531.769531,7044.298828


In [81]:
# Plot the real vs predicted values as a line chart
# YOUR CODE HERE!
stocks.hvplot(
    xlabel='Monthly Progression',
    ylabel='BTC Price',
    title='Actual vs Predicted Bitcoin Closing Prices'
)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

**MODEL ANALYSIS:** 

Predicting closing prices on Bitcoin is not useful here. BTC prices are too volatile for a model to predict. it might be better to predict based on percent change of price of BTC.

One thing to take into consideration when it comes to closing price is that Bitcoin isn't trading based on stock market market time. When it turns 4pm, whatever the price a stock is at, that's the closing price. BTC is traded at all hours of the day, so what is considered a closing price, I assume it's 11:59pm. And it's traded on weekends and holidays. So there is a lot data generated throughout the day to arrive at a closing price. 

Bitcoin in this case, which is prone to huge price fluctuations each day. Building a model to predict on prices alone, most likely will predict the closing price for a target day, to be in line with the closing price of the previous day. So we see in the beginning of our time window predictions have a slow upward trend. BTC crashed in 2018, many small investors lost a lot, so it is not surprising actual prices were below predicted. Investors skeptical to get back in the BTC market withheld investing. But Bitcoin enthusiasm increased and ralied mid-2019, but predictions stayed in par with the slow price trends. 

Just look at the tail end of our allotted window in year 2019. Huge uptick in actual BTC prices, some spikiness and little selloffs at the peak in July 2019, but predictions are much more in line of a slow trend from early part of the time window. The predicitons are using the previous day closing prices from that time to predict a similar closing price for the next day, but that's not how BTC prices actually behave on any given day. Actual BTC prices are well above the predicted values by a difference of 2300. This is because Bitcoin only was just recovering from the 2018 global crash from mass investor selloff taking profits, and this led to optimistic buyers to re-invest in Bitcoin throughout 2019, buying in at the lower dip, raising the prices. Plus, Bitcoin can bought at any time. It does not operate like the stock martket where you can only execute trade orders when the US market is open. And there's limitation as to who in the world can buy into Bitcoin worldwide. 